# Initialization

In [1]:
# import libraries
import os
import sys
import time
import pandas as pd
import numpy as np
from scipy import stats
from scipy.interpolate import CubicSpline
import torch.optim as optim

In [2]:
num_epochs = 20
BATCH_SIZE = 128
learning_rate = 0.001

# Load Data

In [3]:
# load data without header
data1 = pd.read_csv('./ISWC21_data_plus_raw/wetlab_data.csv')
# add header
data1.columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity', 'activity_label_2']
data1.head()

,subject_id,acc_x,acc_y,acc_z,activity,activity_label_2
0,0,0.306563,9.196875,-1.22625,null_class,null_class
1,0,0.306563,9.196875,-1.22625,null_class,null_class
2,0,0.306563,9.196875,-1.22625,null_class,null_class
3,0,0.306563,9.196875,-1.22625,null_class,null_class
4,0,0.306563,9.196875,-1.22625,null_class,null_class


In [4]:
#remove activity label 2 column
data1 = data1.drop(['activity_label_2'], axis=1)
data1.shape

(3163679, 5)

In [5]:
# load data without header
data2 = pd.read_csv('./ISWC21_data_plus_raw/rwhar_data.csv', header=None)
# add header
data2.columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity']
data2.head()

,subject_id,acc_x,acc_y,acc_z,activity
0,0,-9.57434,-2.02733,1.34506,climbing_up
1,0,-9.56479,-1.99597,1.39345,climbing_up
2,0,-9.55122,-1.98445,1.41139,climbing_up
3,0,-9.51335,-1.97557,1.42615,climbing_up
4,0,-9.52959,-1.98187,1.45395,climbing_up


In [6]:
data2.shape

(3200803, 5)

In [7]:
# load data without header
data3 = pd.read_csv('./ISWC21_data_plus_raw/sbhar_data.csv', header=None)
# add header
data3.columns = ['subject_id', 'acc_x', 'acc_y', 'acc_z', 'activity']
data3.head()

,subject_id,acc_x,acc_y,acc_z,activity
0,0,0.443056,0.037500,0.888889,null_class
1,0,0.440278,0.041667,0.880556,null_class
2,0,0.451389,0.043056,0.876389,null_class
3,0,0.456944,0.034722,0.888889,null_class
4,0,0.447222,0.036111,0.888889,null_class


In [8]:
data3.shape

(1122772, 5)

In [9]:
# join all data in one dataframe row-wise
# data = pd.concat([data1, data2, data3], ignore_index=True, axis=0)
data = pd.concat([data1, data3], axis=0)

In [10]:
data.shape

(4286451, 5)

In [11]:
# describe data
data.describe()

,subject_id,acc_x,acc_y,acc_z
count,4.286451e+06,4.286451e+06,4.286451e+06,4.286451e+06
mean,1.156435e+01,-3.142137e+00,-1.027698e+00,2.850134e+00
std,7.311629e+00,4.683064e+00,3.358477e+00,3.758104e+00
min,0.000000e+00,-3.924000e+01,-3.924000e+01,-3.924000e+01
25%,5.000000e+00,-7.050938e+00,-2.759063e+00,2.777778e-03
50%,1.100000e+01,-3.678750e+00,-2.972222e-01,2.759063e+00
75%,1.700000e+01,9.166667e-01,5.930556e-01,6.131250e+00
max,2.900000e+01,3.893344e+01,3.310875e+01,3.893344e+01


In [12]:
# check for missing values
data.isnull().sum()

subject_id    0
acc_x         0
acc_y         0
acc_z         0
activity      0
dtype: int64

In [13]:
data.shape

(4286451, 5)

# Windowing Data

In [14]:
# function for sliding window

def sliding_window_samples(data, samples_per_window, overlap_ratio):
    """
    Return a sliding window measured in number of samples over a data array.

    :param data: input array, can be numpy or pandas dataframe
    :param samples_per_window: window length as number of samples
    :param overlap_ratio: overlap is meant as percentage and should be an integer value
    :return: tuple of windows and indices
    """
    windows = []
    indices = []
    curr = 0
    win_len = int(samples_per_window)
    if overlap_ratio is not None:
        overlapping_elements = int((overlap_ratio / 100) * (win_len))
        if overlapping_elements >= win_len:
            print('Number of overlapping elements exceeds window size.')
            return
    while curr < len(data) - win_len:
        windows.append(data[curr:curr + win_len])
        indices.append([curr, curr + win_len])
        curr = curr + win_len - overlapping_elements
    try:
        result_windows = np.array(windows)
        result_indices = np.array(indices)
    except:
        result_windows = np.empty(shape=(len(windows), win_len, data.shape[1]), dtype=object)
        result_indices = np.array(indices)
        for i in range(0, len(windows)):
            result_windows[i] = windows[i]
            result_indices[i] = indices[i]
    return result_windows, result_indices

In [15]:
sampling_rate = 50
time_window = 2
window_size = sampling_rate * time_window
overlap_ratio = 0

window_data, _ = sliding_window_samples(data, window_size, overlap_ratio)
print(f"shape of window dataset (2 sec with 0% overlap): {window_data.shape}")

shape of window dataset (2 sec with 0% overlap): (42864, 100, 5)


In [16]:
window_data[0]

array([[0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875, -1.22625, 'null_class'],
       [0, 0.3065625, 9.196875,

In [17]:
# remove the label column
window_data = window_data[:, :, :-1]
# window_data = window_data[:, :, :-1]
#remove the subject column
window_data = window_data[:, :, 1:]

In [18]:
window_data[0].shape

(100, 3)

In [19]:
window_data[0]

array([[0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],
       [0.3065625, 9.196875, -1.22625],


# Signal Transformation

In [20]:
def add_random_noise_single_window(data, noise_level=0.1):
    """
    Add random Gaussian noise to a single window of data.

    :param data: Input data with shape (window_size, 3).
    :param noise_level: Standard deviation of the Gaussian noise.
    :return: Noisy data.
    """
    noise = np.random.normal(0, noise_level, data.shape)
    noisy_data = data + noise
    return noisy_data


In [21]:
def random_cropping_single_window(data, crop_ratio=0.8):
    """
    Randomly crop a single window of data and pad it to maintain original shape.

    :param data: Input data with shape (window_size, 3).
    :param crop_ratio: Ratio of the original window size to keep.
    :return: Cropped and padded data.
    """
    window_size = data.shape[0]
    new_size = int(window_size * crop_ratio)
    start = np.random.randint(0, window_size - new_size)
    end = start + new_size
    cropped_data = data[start:end, :]

    # Pad the cropped data to maintain original window size
    padding_size = window_size - new_size
    padding = np.zeros((padding_size, data.shape[1]))
    padded_data = np.vstack((cropped_data, padding))

    return padded_data


In [22]:
def magnitude_warping_single_window(data, warp_factor=0.2):
    """
    Apply magnitude warping to a single window of data.

    :param data: Input data with shape (window_size, 3).
    :param warp_factor: Factor to determine the magnitude of warping.
    :return: Warped data.
    """
    window_size = data.shape[0]
    warped_data = np.copy(data)

    for j in range(3):  # for each axis
        time_points = np.linspace(0, 1, window_size)
        random_points = np.linspace(0, 1, np.random.randint(4, 10))
        warp_values = 1 + np.random.normal(0, warp_factor, random_points.size)
        interpolator = CubicSpline(random_points, warp_values)
        warped_data[:, j] *= interpolator(time_points)

    return warped_data


In [23]:
def time_warping_single_window(data, warp_factor=0.2):
    """
    Apply time warping to a single window of data.

    :param data: Input data with shape (window_size, 3).
    :param warp_factor: Factor to determine the magnitude of time warping.
    :return: Time-warped data.
    """
    window_size = data.shape[0]
    warped_data = np.zeros_like(data)
    time_points = np.linspace(0, 1, window_size)
    random_points = np.sort(np.random.rand(np.random.randint(3, 6)))
    warp_values = np.random.normal(1, warp_factor, random_points.size)
    interpolator = CubicSpline(random_points, warp_values)
    warped_time = interpolator(time_points)
    warped_time -= warped_time.min()
    warped_time /= warped_time.max()
    warped_time *= (window_size - 1)

    for j in range(3):  # for each axis
        interpolator = CubicSpline(np.arange(window_size), data[:, j])
        warped_data[:, j] = interpolator(warped_time)

    return warped_data


In [24]:
# add random noise
noisy_data = add_random_noise_single_window(window_data[0], noise_level=0.1)
print(f"shape of noisy data: {noisy_data.shape}")
# random cropping
cropped_data = random_cropping_single_window(window_data[0], crop_ratio=0.8)
print(f"shape of cropped data: {cropped_data.shape}")
# magnitude warping
warped_data = magnitude_warping_single_window(window_data[0], warp_factor=0.2)
print(f"shape of warped data: {warped_data.shape}")
# time warping
time_warped_data = time_warping_single_window(window_data[0], warp_factor=0.2)
print(f"shape of time warped data: {time_warped_data.shape}")

shape of noisy data: (100, 3)
shape of cropped data: (100, 3)
shape of warped data: (100, 3)
shape of time warped data: (100, 3)


In [25]:
#make a copy of the original data
augmented_data = np.copy(window_data)

# create labels for augmented data
augmented_labels = []


# loop over all windows
for i in range(window_data.shape[0]):
    # choose one number from 0 to 3
    random_number = np.random.randint(0, 4)

    if random_number == 0:
        augmented_data[i] = add_random_noise_single_window(window_data[i], noise_level=0.1)
        augmented_labels.append(0)
    elif random_number == 1:
        augmented_data[i] = random_cropping_single_window(window_data[i], crop_ratio=0.8)
        augmented_labels.append(1)
    elif random_number == 2:
        augmented_data[i] = magnitude_warping_single_window(window_data[i], warp_factor=0.2)
        augmented_labels.append(2)
    else:
        augmented_data[i] = time_warping_single_window(window_data[i], warp_factor=0.2)
        augmented_labels.append(3)

In [26]:
#count distribution of labels
unique, counts = np.unique(augmented_labels, return_counts=True)
print(dict(zip(unique, counts)))

{0: 10553, 1: 10513, 2: 10870, 3: 10928}


In [27]:
print(f"shape of augmented data: {augmented_data.shape}")
print(f"shape of augmented labels: {len(augmented_labels)}")

shape of augmented data: (42864, 100, 3)
shape of augmented labels: 42864


In [28]:
# save augmented data and labels
# np.save('./data_processing/augmented_data_join.npy', augmented_data)
# np.save('./data_processing/augmented_labels_join.npy', augmented_labels)

np.save('./data_processing/augmented_data_join_2.npy', augmented_data) # 2 datasets
np.save('./data_processing/augmented_labels_join_2.npy', augmented_labels) # 2 datasets

# Model Architecture

## CNN Feature

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNFeatureExtractor(nn.Module):
    def __init__(self, num_classes=4):
        super(CNNFeatureExtractor, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 12, 128)  # Adjust the input features according to your final conv layer output
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## TCN

In [30]:
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, dilation, padding):
        super(TemporalBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               stride=stride, padding=0, dilation=dilation)
        self.relu2 = nn.ReLU()
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.relu2(out)
        
        res = x if self.downsample is None else self.downsample(x)

        # Adjusting the length of the residual to match the output
        if out.size(2) != res.size(2):
            desired_length = out.size(2)
            res = res[:, :, :desired_length]

        return self.relu(out + res)


class TCN(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size, dropout=0.2, num_classes=4):
        super(TCN, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size + (dilation_size - 1))]

        self.tcn = nn.Sequential(*layers)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(num_channels[-1], num_classes)

    def forward(self, x):
        x = self.tcn(x)
        x = F.avg_pool1d(x, x.size(2)).squeeze(2)  # Global Average Pooling
        x = self.dropout(x)
        return self.fc(x)

In [31]:
# load augmented data and labels
# augmented_data = np.load('./data_processing/augmented_data_join.npy', allow_pickle=True)
# augmented_labels = np.load('./data_processing/augmented_labels_join.npy', allow_pickle=True)

augmented_data = np.load('./data_processing/augmented_data_join_2.npy', allow_pickle=True)
augmented_labels = np.load('./data_processing/augmented_labels_join_2.npy', allow_pickle=True)

In [32]:
augmented_data.shape

(42864, 100, 3)

In [33]:
augmented_labels.shape

(42864,)

In [34]:
#count distribution of labels
unique, counts = np.unique(augmented_labels, return_counts=True)
print(dict(zip(unique, counts)))

{0: 10553, 1: 10513, 2: 10870, 3: 10928}


In [35]:
# # Convert list of arrays to a single numpy array
# if isinstance(augmented_data, list):
#     augmented_data = np.stack(augmented_data)

# # Ensure the data type is float32
# augmented_data = augmented_data.astype(np.float32)

# # Convert to PyTorch tensors
# augmented_data_tensor = torch.from_numpy(augmented_data)
# augmented_labels_tensor = torch.from_numpy(np.array(augmented_labels)).long()

# # split data into train and test set
# train_size = int(0.8 * augmented_data.shape[0])
# test_size = augmented_data.shape[0] - train_size
# train_data, test_data = torch.utils.data.random_split(augmented_data, [train_size, test_size])
# train_labels, test_labels = torch.utils.data.random_split(augmented_labels, [train_size, test_size])

# # create train and test dataset
# train_dataset = torch.utils.data.TensorDataset(train_data, train_labels)
# test_dataset = torch.utils.data.TensorDataset(test_data, test_labels)

# # create train and test dataloader
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# # check if GPU is available
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# DataLoader Preparation

In [36]:
import torch
from torch.utils.data import DataLoader, TensorDataset, Subset
import numpy as np

# Assuming augmented_data and augmented_labels are numpy arrays
augmented_data = augmented_data.astype(np.float32)

# Convert to PyTorch tensors
augmented_data_tensor = torch.from_numpy(augmented_data)
augmented_labels_tensor = torch.from_numpy(augmented_labels)
#convert labels to long
augmented_labels_tensor = augmented_labels_tensor.long()

# split data into train and test sets
train_size = int(0.8 * len(augmented_data_tensor))
test_size = len(augmented_data_tensor) - train_size

# Creating datasets
dataset = TensorDataset(augmented_data_tensor, augmented_labels_tensor)
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Function to extract tensors from Subset
def extract_subset_data(subset, dataset):
    return dataset.tensors[0][subset.indices], dataset.tensors[1][subset.indices]

# Extract data and labels from train and test sets
train_data, train_labels = extract_subset_data(train_dataset, dataset)
test_data, test_labels = extract_subset_data(test_dataset, dataset)

# create train and test TensorDataset
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

# create DataLoader for train and test sets
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [37]:
#  print the shape of train_loader and test_loader
print(f"shape of train_loader: {len(train_loader)}")
print(f"shape of test_loader: {len(test_loader)}")

shape of train_loader: 268
shape of test_loader: 67


# Train and Test Model

In [38]:
# create training function
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.transpose(1, 2)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    return running_loss / len(train_loader)

In [39]:
# create testing function
def test(model, test_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    #calculate accuracy
    correct = 0
    with torch.no_grad():
        for i, data in enumerate(test_loader):
            inputs, labels = data[0].to(device), data[1].to(device)
            inputs = inputs.transpose(1, 2)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            #calculate accuracy
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
    accuracy = correct / len(test_loader.dataset)
    return running_loss / len(test_loader), accuracy

In [40]:
# create function to train and test model
def train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs):
    train_losses = []
    test_losses = []
    test_accuracies = []
    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        test_loss, test_accuracy = test(model, test_loader, criterion, device)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        print(f"Epoch: {epoch + 1}/{num_epochs}.. Train Loss: {train_loss:.3f}.. Test Loss: {test_loss:.3f}.. Test Accuracy: {test_accuracy:.3f}")
    return train_losses, test_losses, test_accuracies

In [41]:
num_classes = 4

model = CNNFeatureExtractor(num_classes=4)

# move model to GPU if available
model.to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# train and test model
num_epochs = 50
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)



Epoch: 1/50.. Train Loss: 1.338.. Test Loss: 1.286.. Test Accuracy: 0.442
Epoch: 2/50.. Train Loss: 1.214.. Test Loss: 1.137.. Test Accuracy: 0.478
Epoch: 3/50.. Train Loss: 1.075.. Test Loss: 1.031.. Test Accuracy: 0.513
Epoch: 4/50.. Train Loss: 1.007.. Test Loss: 0.991.. Test Accuracy: 0.537
Epoch: 5/50.. Train Loss: 0.979.. Test Loss: 0.972.. Test Accuracy: 0.538
Epoch: 6/50.. Train Loss: 0.963.. Test Loss: 0.958.. Test Accuracy: 0.544
Epoch: 7/50.. Train Loss: 0.951.. Test Loss: 0.946.. Test Accuracy: 0.564
Epoch: 8/50.. Train Loss: 0.941.. Test Loss: 0.937.. Test Accuracy: 0.572
Epoch: 9/50.. Train Loss: 0.931.. Test Loss: 0.927.. Test Accuracy: 0.606
Epoch: 10/50.. Train Loss: 0.922.. Test Loss: 0.917.. Test Accuracy: 0.588
Epoch: 11/50.. Train Loss: 0.913.. Test Loss: 0.909.. Test Accuracy: 0.616
Epoch: 12/50.. Train Loss: 0.903.. Test Loss: 0.898.. Test Accuracy: 0.592
Epoch: 13/50.. Train Loss: 0.894.. Test Loss: 0.889.. Test Accuracy: 0.582
Epoch: 14/50.. Train Loss: 0.883..

In [42]:
# save model
# timestamp = time.strftime("%Y%m%d-%H%M")
# model_name = f"cnn_feature_extractor_join_2_dataset.pt"
# model_name = f"cnn_feature_extractor_join_2_dataset_4_augmentations.pt"
# torch.save(model.state_dict(), f"./models/{model_name}")

In [43]:
# Example usage
num_inputs = 3  # Assuming 3 input channels (x, y, z axes of the accelerometer)
num_channels = [64, 128, 256]  # Example channel sizes for each layer
kernel_size = 8  # Kernel size for temporal convolutions

model = TCN(num_inputs, num_channels, kernel_size, num_classes=4).to(device)

# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.00003)  # Replace lr with your learning rate

# train and test model
num_epochs = 100
train_losses, test_losses, test_accuracies = train_and_test(model, train_loader, test_loader, criterion, optimizer, device, num_epochs)

Epoch: 1/100.. Train Loss: 1.398.. Test Loss: 1.390.. Test Accuracy: 0.243
Epoch: 2/100.. Train Loss: 1.395.. Test Loss: 1.389.. Test Accuracy: 0.247
Epoch: 3/100.. Train Loss: 1.393.. Test Loss: 1.388.. Test Accuracy: 0.246
Epoch: 4/100.. Train Loss: 1.392.. Test Loss: 1.388.. Test Accuracy: 0.252
Epoch: 5/100.. Train Loss: 1.390.. Test Loss: 1.387.. Test Accuracy: 0.253
Epoch: 6/100.. Train Loss: 1.390.. Test Loss: 1.387.. Test Accuracy: 0.256
Epoch: 7/100.. Train Loss: 1.390.. Test Loss: 1.387.. Test Accuracy: 0.257
Epoch: 8/100.. Train Loss: 1.388.. Test Loss: 1.386.. Test Accuracy: 0.264
Epoch: 9/100.. Train Loss: 1.388.. Test Loss: 1.386.. Test Accuracy: 0.260
Epoch: 10/100.. Train Loss: 1.388.. Test Loss: 1.386.. Test Accuracy: 0.256
Epoch: 11/100.. Train Loss: 1.387.. Test Loss: 1.386.. Test Accuracy: 0.261
Epoch: 12/100.. Train Loss: 1.387.. Test Loss: 1.385.. Test Accuracy: 0.260
Epoch: 13/100.. Train Loss: 1.387.. Test Loss: 1.385.. Test Accuracy: 0.260
Epoch: 14/100.. Train

In [44]:
#save model
# timestamp = time.strftime("%Y%m%d-%H%M")
# model_name = f"tcn_join_2_dataset.pt"
# model_name = f"tcn_join_2_dataset_4_augmentations.pt"   
# torch.save(model.state_dict(), f"./models/{model_name}")